In [2]:
!pip install keras-tuner
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

# Load your dataset
data = pd.read_csv('/content/cardio_train.csv', delimiter=';')  # Update the path

# Separate features (X) and target (y)
target_column_name = 'cardio'  # Replace with the actual column name
X = data.drop(columns=[target_column_name])
y = data[target_column_name]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the hyperparameter tuning space
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Input(shape=(X_train_scaled.shape[1],)))

    # Hyperparameter space for number of hidden layers
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i), min_value=32, max_value=256, step=32), activation='relu'))

    model.add(layers.Dense(1, activation='sigmoid'))

    # Hyperparameter space for learning rate
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

# Initialize the RandomSearch tuner
tuner = RandomSearch(build_model,
                     objective='val_accuracy',
                     max_trials=10,
                     directory='tuner_results',
                     project_name='cardio_tuning')

# Search for the best hyperparameters
tuner.search(X_train_scaled, y_train, epochs=10, validation_data=(X_test_scaled, y_test))

# Get the best model
best_model = tuner.get_best_models(1)[0]

# Evaluate the best model on the test data
y_pred = best_model.predict(X_test_scaled)
y_pred_classes = (y_pred > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred_classes)
print(f'Test Accuracy with Hyperparameter Tuning: {accuracy:.4f}')

Trial 10 Complete [00h 01m 23s]
val_accuracy: 0.734499990940094

Best val_accuracy So Far: 0.7409999966621399
Total elapsed time: 00h 08m 48s
438/438 [==============================] - 1s 2ms/step
Test Accuracy with Hyperparameter Tuning: 0.7410
